# Functions for computing the elasticity of a sumset semigroup

## Libraries

Loading the CommutativeMonoids library. You can download with the command 
```properties
$ git clone https://github.com/D-marina/CommutativeMonoids.git
```
The only library use is `integerSmithNormalFormAndApplications` which is in the folder `Class`

In [175]:
import sys
#location of the folder CommutativeMonoids
# we load the following libraies
pathToCommutativeMonoids='../CommutativeMonoids'
sys.path.insert(0,pathToCommutativeMonoids+'/ClassCSemigroup')
sys.path.insert(0,pathToCommutativeMonoids+'/Class')
sys.path.insert(0,pathToCommutativeMonoids+'/ClassAffine')
sys.path.insert(0,pathToCommutativeMonoids+'/CClass')

from integerSmithNormalFormAndApplications import *

from sympy import init_printing, pprint, latex
init_printing() 

from IPython.display import display, Math, Markdown

We load now all the other libraries we also need and define in sympy new symbols

In [149]:
import functools,math
import sympy
import numpy as np
from PyNormaliz import *
from numpy import array
from sympy import symbols,groebner
from fractions import Fraction
import itertools

x1,x2,x3,x4,x5,x6,x7=symbols("x1 x2 x3 x4 x5 x6 x7")
lX=[x1,x2,x3,x4,x5,x6,x7]
y1,y2,y3,y4,y5,y6,y7=symbols("y1 y2 y3 y4 y5 y6 y7")
lY=[y1,y2,y3,y4,y5,y6,y7]
z1,z2,z3,z4,z5,z6,z7=symbols("z1 z2 z3 z4 z5 z6 z7")
lZ=[z1,z2,z3,z4,z5,z6,z7]
w1,w2,w3,w4,w5,w6,w7=symbols("w1 w2 w3 w4 w5 w6 w7")
lW=[w1,w2,w3,w4,w5,w6,w7]

x,y,z,t,u,v,w=symbols("x y z t u v w")

In [150]:
latex1=lambda x: "$"+latex(x)+"$"
def lprint(*lcad):
    '''
    >>> lprint("hola",latex1(x**7),"ccc")
    '''
    display(Markdown(' '.join(lcad)))

## Operators `|sum|` ($+$) and `|mult|` $\otimes$ of sumset semigroups

The operator $+$ and $\otimes$ are defined below

In [151]:
class Infix:
    def __init__(self, function):
        self.function = function
    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))
    def __or__(self, other):
        return self.function(other)
    def __rlshift__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))
    def __rshift__(self, other):
        return self.function(other)
    def __call__(self, value1, value2):
        return self.function(value1, value2)

@Infix
def add(A,B):
    '''Computes the addition of two sets A and B
    >>> [7,9] |add| [0,2,3]
    '''
    C=list(set([a+b for a in A for b in B]))
    C.sort()
    return C

@Infix
def mult(a,A):
    '''Sum with itself a-times the set A
    >>> 3 |mult| [3,4]
    '''
    if a==0:
        return [0]
    elif a==1:
        A=list(set(A))
        A.sort()
        return A
    else:
        return A |add| mult(a-1,A)

## Function to obtain the ideal of a sumset semigroup (`computationIS`)

We define now the functions for computing the presentation of sumset semigroup of the form of Algorithm 1 of [On the ideals of some sumset semigroups](https://arxiv.org/abs/2102.04100). 

The main function if `computationIS` and it also checks if the given list of generators fulfill the requeriments of Algorithm 1 of [On the ideals of some sumset semigroups](https://arxiv.org/abs/2102.04100).

An example of how to use this funcion is 
```properties
>>> computationIS([3],[4],[6,12],[7,10,13],[0,3,6,9])
```

### Auxiliary functions

In [152]:
def computeSets(lsgS):
    '''Computes some auxiliary sets used in functino computationIS
    '''
    lB=[]
    lA=[]
    la=[]
    for S in lsgS:
        if len(S)==1:
            if S not in lB:
                lB.append(S)
        else:
            a0=S[0]
            if a0!=0 and [a0] not in la:
                la.append([a0])
            aux=(a0,[a-a0 for a in S])
            # cuidado con quitar generadores repetidos
            if aux not in lA:
                lA.append(aux)
            else:
                print("Duplicated generator found, removed", S)
    lkA=[S[1][0:2] for S in lA]
    for S in lA:
        if len(S[1])>2:
            lkA.append([0,S[1][2]])
    lkA=[S for i,S in enumerate(lkA) if S not in lkA[:i]]
    return lB,la,lA,lkA

def computeNM(S,lkA):
    '''Computes a decomposition of S in term of the elements of lA
    >>> computeNM([0,2,5,7],[[0,5],[0,2]])
    '''
    for i in range(S[-1]//lkA[0][-1]+1):
        for j in range(S[-1]//lkA[1][-1]+1):
            if S==(i|mult|lkA[0]) |add| (j|mult|lkA[1]):
                return (i,j)

def removeBinomialsHavingVariables(gb,lVariables):
    gb1=[p for p in gb \
         if set(p.args[0].as_powers_dict().keys()).intersection(lVariables)==set()\
         and \
         set(p.args[1].as_powers_dict().keys()).intersection(lVariables)==set()]
    return gb1
            
def isInAnyPolynomial(var,gb):
    lAux=set([])
    for p in gb:
        aux=set(p.args[0].as_powers_dict().keys()).union(set(p.args[1].as_powers_dict().keys()))
        #print(aux)
        lAux=lAux.union(aux)
    #print(lAux)
    return var in lAux

### Function `computationIS`

In [153]:
def computationIS(*sgS, debug=False):
    '''Computes the ideal of the sumset semigroup generated by sgS
    >>> computationIS([3],[4],[6,12],[7,10,13],[0,3,6,9])
    '''
    lB,la,lA,lkA=computeSets(list(sgS))
    
    #aux=[T[1][:2] for T in lA]
    #aux1=[S for i,S in enumerate(aux) if S not in aux[:i]]
    aux1=lkA
    if debug:
        print(aux1)
    if len(aux1)!=2:
        print("We cannot compute the ideal. This case is still not solved")
        return
    Sprime=lB+la+aux1
    S1=lB+la # [[5],[7],[11]]
    
    lnm=[(S[0],S[1],computeNM(S[1],lkA)) for S in lA]
    
    sn=[S[0] for S in lB+la] # [5,7,11]
    a,b=lkA[0][1],lkA[1][1]
    k=math.gcd(a,b)
    a,b=a//k,b//k
    # u = 1 in the natural numbers
    relations=[u**S[0]-lX[j] for j,S in enumerate(lB)]
    relations=relations+[u**S[0]-lW[j] for j,S in enumerate(la)]
    relations=relations+[x**(b-1)*y**(a-1)*(x**b-y**a)]
    
    for j,TT in enumerate(lnm):
        ai,Anm,coord=TT
        if ai!=0:
            pos=[aa[0] for aa in la].index(ai)
            relations=relations+[lZ[j]-lW[pos]*x**coord[0]*y**coord[1]]
        else:
            relations=relations+[lZ[j]-x**coord[0]*y**coord[1]]
    
    gb=groebner(relations,u,x,y,*lW,*lX,*lZ)
    
    #gb1=[p for p in gb \
    #     if set(p.args[0].as_powers_dict().keys()).intersection([u,x,y]+lW)==set()\
    #     and \
    #     set(p.args[1].as_powers_dict().keys()).intersection([u,x,y]+lW)==set()]
    gb1=removeBinomialsHavingVariables(gb,[u,x,y]+lW)
    if debug:
        print("gb1",gb1)
    lVariables=[]
    for var in lX+lZ:
        if isInAnyPolynomial(var,gb1):
            lVariables.append(var)
    if debug:
        print("lVariables",lVariables)
    if gb1!=[]:
        return groebner(gb1,*lVariables) #,lVariables
    else:
        return []

### Example

In [154]:
gb=computationIS([3],[4],[6,12],[7,10,13],[0,3,6,9])
display(gb)

             ⎛⎡  4     3    3   3   2     3    3           2   2    3         
GroebnerBasis⎝⎣x₁  - x₂ , x₁ ⋅x₂ ⋅z₃  - z₂ , x₁ ⋅z₁⋅z₂ - x₂ ⋅z₂ , x₁ ⋅z₁⋅z₃ - 

  2          2   3   3     3       2   2           3    2   2           2     
x₂ ⋅z₂⋅z₃, x₁ ⋅x₂ ⋅z₃  - z₁ ⋅z₃, x₁ ⋅z₁ ⋅z₂ - x₂⋅z₂ , x₁ ⋅z₁ ⋅z₃ - x₂⋅z₂ ⋅z₃, 

     4   2     2          3        4       3        3          2              
x₁⋅x₂ ⋅z₃  - z₁ ⋅z₂, x₁⋅z₁ ⋅z₂ - z₂ , x₁⋅z₁ ⋅z₃ - z₂ ⋅z₃, x₁⋅z₂  - x₂⋅z₁⋅z₂, x

                      5   2        2    4   3   2     5       4   2   3     5 
₁⋅z₂⋅z₃ - x₂⋅z₁⋅z₃, x₂ ⋅z₃  - z₁⋅z₂ , x₂ ⋅z₂ ⋅z₃  - z₁ ⋅z₂, x₂ ⋅z₂ ⋅z₃  - z₁ ⋅

      3   7   2     9       3   6   3     9       2   11   2     13       2   
z₃, x₂ ⋅z₂ ⋅z₃  - z₁ ⋅z₂, x₂ ⋅z₂ ⋅z₃  - z₁ ⋅z₃, x₂ ⋅z₂  ⋅z₃  - z₁  ⋅z₂, x₂ ⋅z₂

10   3     13          4        5       4        4          15   2     17     
  ⋅z₃  - z₁  ⋅z₃, x₂⋅z₁ ⋅z₂ - z₂ , x₂⋅z₁ ⋅z₃ - z₂ ⋅z₃, x₂⋅z₂  ⋅z₃  - z₁  ⋅z₂, 

     14   3     17       21        19   2    2

## Function of obtain the elasticity of a monoid from one of its ideals (`elasticity`)

### Auxiliary functions

Now, we define three auxiliary funcions `fraccionDeGenerador` and `binomio2Zn` used in the function `elasticity`. The last function computes the elasticity of a monoid from one of its defining ideals.

In [155]:
def fraccionDeGenerador(gen):
    dim=len(gen)//2
    #print(gen,dim)
    return Fraction(sum(gen[:dim]),sum(gen[dim:]))

def binomio2Zn(binomio,lV):
    #binomio=G4[0]
    s1,s2=binomio.args[0].as_powers_dict(),binomio.args[1].as_powers_dict()
    c=[]
    c1=[]
    c2=[]
    #print(s1)
    for v in lV:
        c1.append(s1[v])
    for v in lV:
        c2.append(s2[v])
    return [c1,c2]

We have the function elasticity

In [156]:
lVariables=gb.args[1]
tuplasGB=[binomio2Zn(b,lVariables) for b in gb]
genM=[]
for c1,c2 in tuplasGB:
    genM.append(array(c1)-array(c2))

eqGenM=generatorsToEquations(genM)

numEqs=eqGenM[0].shape[0]
filas=[list(eqGenM[0][i,:]) for i in range(numEqs)]


print(eqGenM)
print(filas)

(Matrix([
[0, 0,  -2,  -2,  -3],
[3, 4, -16, -15, -33]]), Matrix([
[0],
[0]]))
[[0, 0, -2, -2, -3], [3, 4, -16, -15, -33]]


In [157]:
def computeEquationsOfMFromGB(gb):
    lVariables=gb.args[1]
    tuplasGB=[binomio2Zn(b,lVariables) for b in gb]
    genM=[]
    for c1,c2 in tuplasGB:
        genM.append(array(c1)-array(c2))

    eqGenM=generatorsToEquations(genM)
    
    numEqs=eqGenM[0].shape[0]
    filas=[list(eqGenM[0][i,:]) for i in range(numEqs)]
    
    return filas

### Function `elasticity`

In [158]:
def elasticity(gb,verbose=False):
    '''Computes the elasticity of a monoid from one of its presetations.
    It returns the value of the elasticity and the elements of gb that reach the elasticity
    >>> rho,lTElasticity=elasticity(gb)
    '''
    #lVariables=gb.args[1]
    #tuplasGB=[binomio2Zn(b,lVariables) for b in gb]
    #genM=[]
    #for c1,c2 in tuplasGB:
    #    genM.append(array(c1)-array(c2))
    #eqGenM=generatorsToEquations(genM)
    
    #numEqs=eqGenM[0].shape[0]
    #filas=[list(eqGenM[0][i,:]) for i in range(numEqs)]

    filas=computeEquationsOfMFromGB(gb)
    
    coneSR=Cone(equations=filas)
    cSRHB=coneSR.HilbertBasis()
    if verbose:
        print("Cono positivo M:",cSRHB)
    if cSRHB!=[]:
        if verbose:
            print("The semigroup is not strongly reduced")
        return "\infty",[]
    
    
    
    #print("filas",filas)

    ecuacionesAmenosA=[filas[i]+[-a for a in filas[i]] for i in range(len(filas))]
    monoide=Cone(equations=ecuacionesAmenosA)
    sistemaGenMonoide=monoide.HilbertBasis()
    
    dim=len(sistemaGenMonoide[0])//2
    elasticidad=max([fraccionDeGenerador(gen) for gen in sistemaGenMonoide])
    
    elasticidad

    Rs=[gen for gen in sistemaGenMonoide \
     if fraccionDeGenerador(gen)==elasticidad]
    
    if verbose:
        #print("Order of variables:",variablesGB(gb))
        #print(tuplasGB)
        print("Equations of M")
        print(filas)
        #print(eqGenM[0],"\n")
        #print("Matrix (A|-A)")
        #print(ecuacionesAmenosA)

    
    return(elasticidad,Rs)
    

### Example

One example of how to use this function is the following.

In [159]:
sgS=[[3],[4],[6,12],[7,10,13],[0,3,6,9]]
gb=computationIS(*sgS)
rho,lTElasticity=elasticity(gb,verbose=True)

print("Variables used in the Groebner basis:",gb.args[1])
print("Elasticity of S:",rho)
print("Pairs of atoms of A(I_M) that reach the elasticity:",lTElasticity)

Cono positivo M: []
Equations of M
[[0, 0, -2, -2, -3], [3, 4, -16, -15, -33]]
Variables used in the Groebner basis: (x1, x2, z1, z2, z3)
Elasticity of S: 3
Pairs of atoms of A(I_M) that reach the elasticity: [[7, 0, 0, 0, 2, 0, 0, 0, 3, 0]]


Since the monomial $x1^7 z3^2$ is reduced to $z2^3$, the element $7 \otimes \{3\} + 2 \otimes \{0,3,6,9\}=3 \otimes \{7,10,13\}$ and reachs the elasticity. 
So, the monoids $S$ has acceptable elasticity.

In [160]:
len(gb.args[0])
len([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

25

In [161]:
gb.reduce(x1**7*z3**2)

⎛⎡  3   2                                                                     
⎝⎣x₁ ⋅z₃ , 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

   ⎤    3⎞
, 0⎦, z₂ ⎠

## Algorithm to decide if a semigroups has acceptable elasticity (`hasAcceptableElasticity`)

From the work [Atomic Commutative Monoids and Their Elasticity
](https://doi-org.bibezproxy.uca.es/10.1007/s00233-002-0022-4), we can check from a presetation of a monoid if it has acceptable elasticity or not.

### Auxiliary functions

We need a function to compute the Archimedean components of a monoid from one a system of generators of $I_S$

In [162]:
def projection01(l):
    return [(lambda x:0 if x==0 else 1)(a) for a in l]

projection01([5,0,1,0,0,3])

def supreme(t1,t2):
    return [max(x,y) for x,y in zip(t1,t2)]
def infimum(t1,t2):
    return [min(x,y) for x,y in zip(t1,t2)]
def elementsBetween(t1,t2):
    s=[]
    for tt in itertools.product([0,1],repeat=len(t1)):
        if infimum(tt,t1)==t1 and supreme(tt,t2)==t2:
            s.append(tt)
    return s

def areInTheSameACAux(Grho,vv1,vv2):
    for aa,bb in Grho:
        if aa==vv1:
            vv1=bb
        if aa==vv2:
            vv2=bb
    if vv1==vv2:
        return True
    return False

def ArchimedeanComponents(gb, debug=False):
    lV=gb.args[1]
    lLists=[binomio2Zn(p,lV) for p in gb]
    l01=[(projection01(a),projection01(b)) for a,b in lLists]+\
        [(projection01(b),projection01(a)) for a,b in lLists]
    #print(lLists)
    if debug:
        print(l01,"\n")
    latt=itertools.product([0,1],repeat=len(lV))
    #print(list(latt))
    
    Grho=[]
    for vv in latt:
        #print(vv)
        menores=[tt for tt in l01 if tt[0]==infimum(tt[0],list(vv))]
        #print(menores,"\n")
        for a,b in menores:
            for i,vb in enumerate(b):
                if vb==1 and vv[i]==0:
                    copiavv1=list(vv)
                    copiavv1[i]=1
                    if (vv,copiavv1) not in Grho:
                        Grho.append((list(vv),copiavv1))
    if debug:
        print("Grho:",Grho,"\n")
                        
    lACcomponents=[]
    for vv in itertools.product([0,1],repeat=len(lV)):
        vv=list(vv)
        if debug:
            print("lACcomponents:",lACcomponents)
            print(vv)
        if lACcomponents==[]:
            lACcomponents.append([vv])
        else:
            encontrado=False
            i=0
            while not encontrado and i<len(lACcomponents):
                lAux=lACcomponents[i]
                primero=lAux[0]                
                vt=areInTheSameACAux(Grho,vv,primero)
                if debug:
                    print("\tvv cc",vv,primero,vt)
                if vt:
                    encontrado=True
                    lAux.append(vv)
                    lACcomponents[i]=lAux
                i=i+1 
            if not encontrado:
                lACcomponents.append([vv])
    if debug:
        print("lACcomponents",lACcomponents)
    
    return lACcomponents #,Grho

In [163]:
sgS=[[3],[4],[6,12],[7,10,13],[0,3,6,9]]
gb=computationIS(*sgS)
print(gb.args[0])
print(gb.args[1])
AC=ArchimedeanComponents(gb)
print("Archimedean components of S:")
for i,arch in enumerate(AC):
    print(i,"-->",arch)
print("The maximum number of Archimedean component \
for a monoid generated by 5 elements is {}.".format(2**5))
print("The monoid S has only",len(AC),"Archimedean components")

(x1**4 - x2**3, x1**3*x2**3*z3**2 - z2**3, x1**3*z1*z2 - x2**2*z2**2, x1**3*z1*z3 - x2**2*z2*z3, x1**2*x2**3*z3**3 - z1**3*z3, x1**2*z1**2*z2 - x2*z2**3, x1**2*z1**2*z3 - x2*z2**2*z3, x1*x2**4*z3**2 - z1**2*z2, x1*z1**3*z2 - z2**4, x1*z1**3*z3 - z2**3*z3, x1*z2**2 - x2*z1*z2, x1*z2*z3 - x2*z1*z3, x2**5*z3**2 - z1*z2**2, x2**4*z2**3*z3**2 - z1**5*z2, x2**4*z2**2*z3**3 - z1**5*z3, x2**3*z2**7*z3**2 - z1**9*z2, x2**3*z2**6*z3**3 - z1**9*z3, x2**2*z2**11*z3**2 - z1**13*z2, x2**2*z2**10*z3**3 - z1**13*z3, x2*z1**4*z2 - z2**5, x2*z1**4*z3 - z2**4*z3, x2*z2**15*z3**2 - z1**17*z2, x2*z2**14*z3**3 - z1**17*z3, z1**21*z2 - z2**19*z3**2, z1**21*z3 - z2**18*z3**3)
(x1, x2, z1, z2, z3)
Archimedean components of S:
0 --> [[0, 0, 0, 0, 0]]
1 --> [[0, 0, 0, 0, 1]]
2 --> [[0, 0, 0, 1, 0], [0, 0, 0, 1, 1], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0], [0, 0, 1, 1, 1], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1], [0, 1, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0], [1, 0, 0, 1, 

In [164]:
def supportIncluded(vect1,vect2):
    '''
    >>> supportIncluded([0,0,3],[1,0,1])
    '''
    for i,j in zip(vect1,vect2):
        if i!=0 and j==0:
            return False
    return True


def hasAcceptableElasticity1(gb,Rs,showTuple=False):
    '''From one of its presentations, computes if the elasticity of a monoid is acceptable
    >>> hasAcceptableElasticity(gb,lTElasticity)
    '''
    tuplasGB=[binomio2Zn(b,variablesGB(gb)) for b in gb]
    ts=[vv[0]+vv[1] for vv in tuplasGB]
    for vv in ts:
        for ww in Rs:
            if supportIncluded(vv,ww):
                if showTuple:
                    print("Support of",vv,"included in the support of",ww)
                return True
    return False

In [165]:
def computationOfC(lTElasticity):
    projLTE=[projection01(vv) for vv in lTElasticity]

    res = []
    [res.append(x) for x in projLTE if x not in res]

    projLTE=res
    #print(projLTE)
    lOld=0
    while lOld!=len(projLTE):
        lOld=len(projLTE)
        listAux=list(projLTE)
        for i,A in enumerate(listAux):
            for j,B in enumerate(listAux[i+1:]):
                C=supreme(A,B)
                #print("C",C)
                if not C in projLTE:
                    projLTE.append(C)
    
    dim2=len(lTElasticity[0])//2
    aux=[supreme(vv[0:dim2],vv[dim2:]) for vv in projLTE]

                    
    return(projLTE,aux)

In [166]:
def areInTheSameAC(ac1,ac2,AC):
    for cc in AC:
        if ac1 in cc:
            if ac2 in cc:
                return True
            else:
                return False

### Function `hasAcceptableElasticity`

In [167]:
def hasAcceptableElasticity(gb1, debug=False):
    rho,lTElasticity=elasticity(gb1)
    AC=ArchimedeanComponents(gb1)
    Cmonoid,Caux=computationOfC(lTElasticity)
    
    #for cc in Cmonoid:
    #    izq=cc[:len(cc)//2]
    #    der=cc[len(cc)//2:]
    #    tt=areInTheSameAC(izq,der,AC)
        
    Cset=[cc for cc in Cmonoid if areInTheSameAC(cc[:len(cc)//2],cc[len(cc)//2:],AC)]
    if Cset==[]:
        print("S has not acceptable elasticity")
        return
    
    lVV=gb1.args[1]
    if debug:
        print(lVV,"\n")
    for tt1 in Cset:
        tam=len(tt1)
        print("tt1",tt1)
        izq=tt1[:tam//2]
        der=tt1[tam//2:]
        if debug:
            print(izq,der)
        tt=supreme(izq,der)

        posIzq=[i for i,vp in enumerate(izq) if vp!=0]
        posDer=[i for i,vp in enumerate(der) if vp!=0]
        goodPos=posIzq+posDer
        goodPos.sort()
        tamGood=len(goodPos)
        if debug:
            print("Good positions",goodPos)
        supportA=[izq[i] for i in goodPos]
        supportB=[der[i] for i in goodPos]
        if debug:
            print("supportA supportB",supportA,supportB)
        print(tt)
        lVaux1=[var for i,var in enumerate(lVV) if tt[i]==0]
        lVaux2=[var for i,var in enumerate(lVV) if tt[i]==1]
        if debug:
            print("lVaux2",lVaux2)
        gbT=groebner(gb1,lVaux1+lVaux2)
        print(gbT)
        #print(removeBinomialsHavingVariables(gbT,lVaux1))
        gbTAux=groebner( removeBinomialsHavingVariables(gbT,lVaux1) )
        print(gbTAux)
        ACTaux=ArchimedeanComponents(gbTAux)
        print(ACTaux)
        eqM=computeEquationsOfMFromGB(gbTAux)
        print("equations",eqM)
        cone=Cone(equations=[l+[-a for a in l] for l in eqM])
        base=cone.HilbertBasis()
        if debug:
            print("base",base)
            print()

        genInclAB=[g for g in base if supportIncluded(g[:tamGood],supportA) and supportIncluded(g[tamGood:],supportB)]    
        print("genInclAB",genInclAB)
        laux=[0 for i in range(tamGood)]
        for tg in genInclAB:
            supr=supreme( projection01(tg[:tamGood]), projection01(tg[tamGood:]) )
            laux=supreme(supr,laux)
        if laux==[1 for i in range(tamGood)]:
            print("S has acceptable elasticity")
        else:
            print("S has NOT acceptable elasticity")

### Example of how to use the function `hasAcceptableElasticity`

In [168]:
gb=computationIS([3],[4],[6,12],[7,10,13],[0,3,6,9])
#lprint("Groebner basis:")
#lprint(gb.args[0])
#lprint("Variables used:")
#lprint(gb.args[1])
display(gb)
hasAcceptableElasticity(gb,debug=True)

             ⎛⎡  4     3    3   3   2     3    3           2   2    3         
GroebnerBasis⎝⎣x₁  - x₂ , x₁ ⋅x₂ ⋅z₃  - z₂ , x₁ ⋅z₁⋅z₂ - x₂ ⋅z₂ , x₁ ⋅z₁⋅z₃ - 

  2          2   3   3     3       2   2           3    2   2           2     
x₂ ⋅z₂⋅z₃, x₁ ⋅x₂ ⋅z₃  - z₁ ⋅z₃, x₁ ⋅z₁ ⋅z₂ - x₂⋅z₂ , x₁ ⋅z₁ ⋅z₃ - x₂⋅z₂ ⋅z₃, 

     4   2     2          3        4       3        3          2              
x₁⋅x₂ ⋅z₃  - z₁ ⋅z₂, x₁⋅z₁ ⋅z₂ - z₂ , x₁⋅z₁ ⋅z₃ - z₂ ⋅z₃, x₁⋅z₂  - x₂⋅z₁⋅z₂, x

                      5   2        2    4   3   2     5       4   2   3     5 
₁⋅z₂⋅z₃ - x₂⋅z₁⋅z₃, x₂ ⋅z₃  - z₁⋅z₂ , x₂ ⋅z₂ ⋅z₃  - z₁ ⋅z₂, x₂ ⋅z₂ ⋅z₃  - z₁ ⋅

      3   7   2     9       3   6   3     9       2   11   2     13       2   
z₃, x₂ ⋅z₂ ⋅z₃  - z₁ ⋅z₂, x₂ ⋅z₂ ⋅z₃  - z₁ ⋅z₃, x₂ ⋅z₂  ⋅z₃  - z₁  ⋅z₂, x₂ ⋅z₂

10   3     13          4        5       4        4          15   2     17     
  ⋅z₃  - z₁  ⋅z₃, x₂⋅z₁ ⋅z₂ - z₂ , x₂⋅z₁ ⋅z₃ - z₂ ⋅z₃, x₂⋅z₂  ⋅z₃  - z₁  ⋅z₂, 

     14   3     17       21        19   2    2

(x1, x2, z1, z2, z3) 

tt1 [1, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[1, 0, 0, 0, 1] [0, 0, 0, 1, 0]
Good positions [0, 3, 4]
supportA supportB [1, 0, 1] [0, 1, 0]
[1, 0, 0, 1, 1]
lVaux2 [x1, z2, z3]
GroebnerBasis([-x1**4 + x2**3, x1**4*x2**2*z3**2 - z1*z2**2, -x1**3*z1*z2 + x2**2*z2**2, -x1**3*z1*z3 + x2**2*z2*z3, -x1*z2**2 + x2*z1*z2, -x1*z2*z3 + x2*z1*z3, x1**5*x2*z3**2 - z1**2*z2, -x1**2*z1**2*z2 + x2*z2**3, -x1**2*z1**2*z3 + x2*z2**2*z3, -x1**6*z2*z3**2 + z1**3*z2, -x1**6*z3**3 + z1**3*z3, x1**7*z3**2 - z2**3], x2, z1, x1, z2, z3, domain='ZZ', order='lex')
GroebnerBasis([x1**7*z3**2 - z2**3], x1, z2, z3, domain='ZZ', order='lex')
[[[0, 0, 0]], [[0, 0, 1]], [[0, 1, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0], [1, 1, 1]], [[1, 0, 0]]]
equations [[0, -2, -3], [1, -7, -14]]
base [[0, 0, 1, 0, 0, 1], [0, 1, 0, 0, 1, 0], [0, 3, 0, 7, 0, 2], [1, 0, 0, 1, 0, 0], [7, 0, 2, 0, 3, 0]]

genInclAB [[7, 0, 2, 0, 3, 0]]
S has acceptable elasticity


## Examples

### Example 1

The following semigroup is not cancellative (see the Groebner basis obtained, there are variables that are not cancelled in some binomials)

In [173]:
vc11=((3 |mult| [0,3]) |add| [0,5]) |add| [4] 
vc12=((2 |mult| [0,3]) |add| (2 |mult| [0,5])) |add| [2] 
vc13=((2 |mult| [0,3]) |add| (4 |mult| [0,5])) |add| [7] 
display(vc11)
display(vc12)
display(vc13)

[4, 7, 9, 10, 12, 13, 15, 18]

[2, 5, 7, 8, 10, 12, 13, 15, 18]

[7, 10, 12, 13, 15, 17, 18, 20, 22, 23, 25, 27, 28, 30, 33]

In [174]:
gb1=computationIS([3],[7],vc11,vc12,vc13,debug=True)

[[0, 3], [0, 5]]


KeyboardInterrupt: 

In [120]:
display(gb1.args[0])
display(gb1.args[1])

⎛  7     3    6   3     3          5   6     3   2   2    4   5     2   2    4
⎝x₁  - x₂ , x₁ ⋅z₁  - x₂ ⋅z₂⋅z₃, x₁ ⋅z₁  - x₂ ⋅z₂ ⋅z₃ , x₁ ⋅z₂  - z₁ ⋅z₃ , x₁ 

   3     3      4    3   2   2     3   5    3      4     3    2   5        3  
⋅z₃  - x₂ ⋅z₁⋅z₂ , x₁ ⋅z₁ ⋅z₃  - x₂ ⋅z₂ , x₁ ⋅z₁⋅z₂  - z₃ , x₁ ⋅z₁ ⋅z₃ - x₂ ⋅z

 6    2   4   3     4       8     3   7       7   2     5               3     
₂ , x₁ ⋅z₁ ⋅z₂  - z₃ , x₁⋅z₁  - x₂ ⋅z₂ , x₁⋅z₁ ⋅z₂  - z₃ , x₁⋅z₂⋅z₃ - z₁ , x₁⋅

  6     3   2   8    3   9        5    3   8        11    3   7   7     21    
z₃  - x₂ ⋅z₁ ⋅z₂ , x₂ ⋅z₂  - z₁⋅z₃ , x₂ ⋅z₂ ⋅z₃ - z₁  , x₂ ⋅z₂ ⋅z₃  - z₁  , x₂

3   6   13     31    3   5   19     41    3   4   25     51    3   3   31     
 ⋅z₂ ⋅z₃   - z₁  , x₂ ⋅z₂ ⋅z₃   - z₁  , x₂ ⋅z₂ ⋅z₃   - z₁  , x₂ ⋅z₂ ⋅z₃   - z₁

61    3   2   37     71    3      43     81    3   49     91    10        6⎞
  , x₂ ⋅z₂ ⋅z₃   - z₁  , x₂ ⋅z₂⋅z₃   - z₁  , x₂ ⋅z₃   - z₁  , z₁  ⋅z₂ - z₃ ⎠

(x₁, x₂, z₁, z₂, z₃)

In [75]:
elasticidad1,Rs1=elasticity(gb1,verbose=True)
print("Elasticity:",elasticidad1)
print(Rs1)

Cono positivo M: []
Equations of M
[[39, 91, 3, -30, 0], [84, 196, 7, -64, 1]]
Elasticity: 23/8
[[10, 0, 0, 13, 0, 0, 0, 0, 0, 8]]


In [76]:
hasAcceptableElasticity(gb1,debug=True)

(x1, x2, z1, z2, z3) 

tt1 [1, 0, 0, 1, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 1, 0] [0, 0, 0, 0, 1]
Good positions [0, 3, 4]
supportA supportB [1, 1, 0] [0, 0, 1]
[1, 0, 0, 1, 1]
lVaux2 [x1, z2, z3]
GroebnerBasis([-x1**7 + x2**3, -x1*z2*z3 + z1**3, -x1**4*z2**5 + z1**2*z3**2, x1**3*z1*z2**4 - z3**3, -x1**7*z2**9 + z1*z3**5, x1**10*z2**13 - z3**8], x2, z1, x1, z2, z3, domain='ZZ', order='lex')
GroebnerBasis([x1**10*z2**13 - z3**8], x1, z2, z3, domain='ZZ', order='lex')
[[[0, 0, 0]], [[0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0], [1, 1, 1]], [[0, 1, 0]], [[1, 0, 0]]]
equations [[0, -8, -13], [1, 30, 50]]
base [[0, 0, 1, 0, 0, 1], [0, 0, 8, 10, 13, 0], [0, 1, 0, 0, 1, 0], [1, 0, 0, 1, 0, 0], [10, 13, 0, 0, 0, 8]]

genInclAB [[10, 13, 0, 0, 0, 8]]
S has acceptable elasticity


### Example 2

In [ ]:
vec21=( (3 |mult| [0,4]) |add| (2 |mult| [0,5]) ) |add| [8]
vec22=((2 |mult| [0,4]) |add| [0,5])|add| [7]
vec23=(3 |mult| [0,4]) |add| (3 |mult| [0,5])
vec24=( (4 |mult| [0,4]) |add| (1 |mult| [0,5]) ) |add| [2]
vec21,vec22,vec23,vec24

In [ ]:
gb2=computationIS(vec21,vec22,vec23,vec24,
                  debug=True)

In [119]:
display(gb2.args[0])
display(gb2.args[1])

⎛  31     34   5   5    18        18   9    13   4     16   6    8   17     14
⎝z₁   - z₂  ⋅z₃ ⋅z₄ , z₁  ⋅z₃ - z₂  ⋅z₄ , z₁  ⋅z₄  - z₂  ⋅z₃ , z₁ ⋅z₄   - z₂  

   13    5   7     2   13    3   30     12   20    2   10   27     43       32
⋅z₃  , z₁ ⋅z₃  - z₂ ⋅z₄  , z₁ ⋅z₄   - z₂  ⋅z₃  , z₁ ⋅z₂  ⋅z₃   - z₄  , z₁⋅z₂  

   74     116       73     22   47    54   121     189⎞
⋅z₃   - z₄   , z₁⋅z₄   - z₂  ⋅z₃  , z₂  ⋅z₃    - z₄   ⎠

(z₁, z₂, z₃, z₄)

In [110]:
elasticidad2,Rs2=elasticidad,Rs=elasticity(gb2,verbose=True)
print()
print(elasticidad2)
print(Rs2)

Cono positivo M: []
Equations of M
[[-50, 0, -189, -121], [-2356, 1, -8910, -5704]]

174/121
[[0, 124, 0, 50, 121, 0, 0, 0]]


In [81]:
hasAcceptableElasticity(gb2,debug=True)

(z1, z2, z3, z4) 

tt1 [0, 1, 0, 1, 1, 0, 0, 0]
[0, 1, 0, 1] [1, 0, 0, 0]
Good positions [0, 1, 3]
supportA supportB [0, 1, 1] [1, 0, 0]
[1, 1, 0, 1]
lVaux2 [z1, z2, z4]
GroebnerBasis([z1**2*z2**10*z3**27 - z4**43, -z1**3*z4**30 + z2**12*z3**20, -z1**8*z4**17 + z2**14*z3**13, z1**5*z3**7 - z2**2*z4**13, -z1**13*z4**4 + z2**16*z3**6, -z1**31 + z2**34*z3**5*z4**5, -z1**49 + z2**52*z3**4*z4**14, -z1**67 + z2**70*z3**3*z4**23, -z1**85 + z2**88*z3**2*z4**32, z1**18*z3 - z2**18*z4**9, -z1**103 + z2**106*z3*z4**41, z1**121 - z2**124*z4**50], z3, z1, z2, z4, domain='ZZ', order='lex')
GroebnerBasis([z1**121 - z2**124*z4**50], z1, z2, z4, domain='ZZ', order='lex')
[[[0, 0, 0]], [[0, 0, 1]], [[0, 1, 0]], [[0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]]]
equations [[0, 25, -62], [-2, 242, -605]]
base [[0, 0, 1, 0, 0, 1], [0, 1, 0, 0, 1, 0], [0, 124, 50, 121, 0, 0], [1, 0, 0, 1, 0, 0], [121, 0, 0, 0, 124, 50]]

genInclAB [[0, 124, 50, 121, 0, 0]]
S has acceptable elasticity


### Example 3

The monoid considered is $\langle \{3\},\{4\},\{6,12\},\{7,10,13\},\{0,3,6,9\} \rangle$

In [113]:
gb3=computationIS([3],[4],[6,12],[7,10,13],[0,3,6,9])

In [107]:
display(gb3.args[0])
display(gb3.args[1])

⎛  4     3    3   3   2     3    3           2   2    3           2          2
⎝x₁  - x₂ , x₁ ⋅x₂ ⋅z₃  - z₂ , x₁ ⋅z₁⋅z₂ - x₂ ⋅z₂ , x₁ ⋅z₁⋅z₃ - x₂ ⋅z₂⋅z₃, x₁ 

   3   3     3       2   2           3    2   2           2          4   2    
⋅x₂ ⋅z₃  - z₁ ⋅z₃, x₁ ⋅z₁ ⋅z₂ - x₂⋅z₂ , x₁ ⋅z₁ ⋅z₃ - x₂⋅z₂ ⋅z₃, x₁⋅x₂ ⋅z₃  - z

 2          3        4       3        3          2                            
₁ ⋅z₂, x₁⋅z₁ ⋅z₂ - z₂ , x₁⋅z₁ ⋅z₃ - z₂ ⋅z₃, x₁⋅z₂  - x₂⋅z₁⋅z₂, x₁⋅z₂⋅z₃ - x₂⋅z

        5   2        2    4   3   2     5       4   2   3     5       3   7   
₁⋅z₃, x₂ ⋅z₃  - z₁⋅z₂ , x₂ ⋅z₂ ⋅z₃  - z₁ ⋅z₂, x₂ ⋅z₂ ⋅z₃  - z₁ ⋅z₃, x₂ ⋅z₂ ⋅z₃

2     9       3   6   3     9       2   11   2     13       2   10   3     13 
  - z₁ ⋅z₂, x₂ ⋅z₂ ⋅z₃  - z₁ ⋅z₃, x₂ ⋅z₂  ⋅z₃  - z₁  ⋅z₂, x₂ ⋅z₂  ⋅z₃  - z₁  ⋅

         4        5       4        4          15   2     17          14   3   
z₃, x₂⋅z₁ ⋅z₂ - z₂ , x₂⋅z₁ ⋅z₃ - z₂ ⋅z₃, x₂⋅z₂  ⋅z₃  - z₁  ⋅z₂, x₂⋅z₂  ⋅z₃  - 

  17       21        19   2    21        18   

(x₁, x₂, z₁, z₂, z₃)

In [85]:
rho3,lTElasticity3=elasticity(gb3)
rho3,lTElasticity3

(Fraction(3, 1), [[7, 0, 0, 0, 2, 0, 0, 0, 3, 0]])

In [86]:
hasAcceptableElasticity(gb3,debug=True)

(x1, x2, z1, z2, z3) 

tt1 [1, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[1, 0, 0, 0, 1] [0, 0, 0, 1, 0]
Good positions [0, 3, 4]
supportA supportB [1, 0, 1] [0, 1, 0]
[1, 0, 0, 1, 1]
lVaux2 [x1, z2, z3]
GroebnerBasis([-x1**4 + x2**3, x1**4*x2**2*z3**2 - z1*z2**2, -x1**3*z1*z2 + x2**2*z2**2, -x1**3*z1*z3 + x2**2*z2*z3, -x1*z2**2 + x2*z1*z2, -x1*z2*z3 + x2*z1*z3, x1**5*x2*z3**2 - z1**2*z2, -x1**2*z1**2*z2 + x2*z2**3, -x1**2*z1**2*z3 + x2*z2**2*z3, -x1**6*z2*z3**2 + z1**3*z2, -x1**6*z3**3 + z1**3*z3, x1**7*z3**2 - z2**3], x2, z1, x1, z2, z3, domain='ZZ', order='lex')
GroebnerBasis([x1**7*z3**2 - z2**3], x1, z2, z3, domain='ZZ', order='lex')
[[[0, 0, 0]], [[0, 0, 1]], [[0, 1, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0], [1, 1, 1]], [[1, 0, 0]]]
equations [[0, -2, -3], [1, -7, -14]]
base [[0, 0, 1, 0, 0, 1], [0, 1, 0, 0, 1, 0], [0, 3, 0, 7, 0, 2], [1, 0, 0, 1, 0, 0], [7, 0, 2, 0, 3, 0]]

genInclAB [[7, 0, 2, 0, 3, 0]]
S has acceptable elasticity


### Example 4

In this example the monoid is $\langle \{3\},\{4\},\{6,12\},\{7,10,13\} \rangle$

In [169]:
gb4=computationIS([3],[4],[6,12],[7,10,13])
display(gb4.args[0])
display(gb4.args[1])

⎛  4     3    3           2   2    2   2           3       3        4       2 
⎝x₁  - x₂ , x₁ ⋅z₁⋅z₂ - x₂ ⋅z₂ , x₁ ⋅z₁ ⋅z₂ - x₂⋅z₂ , x₁⋅z₁ ⋅z₂ - z₂ , x₁⋅z₂  

                 4        5⎞
- x₂⋅z₁⋅z₂, x₂⋅z₁ ⋅z₂ - z₂ ⎠

(x₁, x₂, z₁, z₂)

In [170]:
rho4,lTElasticity4=elasticity(gb4)
print(rho4)
print(lTElasticity4)

4/3
[[1, 0, 3, 0, 0, 0, 0, 3], [2, 0, 2, 0, 0, 1, 0, 2], [3, 0, 1, 0, 0, 2, 0, 1], [4, 0, 0, 0, 0, 3, 0, 0]]


In [171]:
AC=ArchimedeanComponents(gb4)
for i,component in enumerate(AC):
    print(component)

[[0, 0, 0, 0]]
[[0, 0, 0, 1], [0, 0, 1, 1], [0, 1, 0, 1], [0, 1, 1, 1], [1, 0, 0, 1], [1, 0, 1, 1], [1, 1, 0, 1], [1, 1, 1, 1]]
[[0, 0, 1, 0]]
[[0, 1, 0, 0], [1, 0, 0, 0], [1, 1, 0, 0]]
[[0, 1, 1, 0], [1, 0, 1, 0], [1, 1, 1, 0]]


In [172]:
hasAcceptableElasticity(gb4,debug=True)

(x1, x2, z1, z2) 

tt1 [1, 0, 0, 0, 0, 1, 0, 0]
[1, 0, 0, 0] [0, 1, 0, 0]
Good positions [0, 1]
supportA supportB [1, 0] [0, 1]
[1, 1, 0, 0]
lVaux2 [x1, x2]
GroebnerBasis([x1*z1**3*z2 - z2**4, x1**2*z1**2*z2 - x2*z2**3, x1**3*z1*z2 - x2**2*z2**2, -x1*z2**2 + x2*z1*z2, x1**4 - x2**3], z1, z2, x1, x2, domain='ZZ', order='lex')
GroebnerBasis([x1**4 - x2**3], x1, x2, domain='ZZ', order='lex')
[[[0, 0]], [[0, 1], [1, 0], [1, 1]]]
equations [[3, 4]]
base [[0, 1, 0, 1], [0, 3, 4, 0], [1, 0, 1, 0], [4, 0, 0, 3]]

genInclAB [[4, 0, 0, 3]]
S has acceptable elasticity


### Example 5

In [211]:
v1= (2 |mult| [0,3]) |add| (3 |mult| [0,4]) |add| [7]
gb5=computationIS([0,3],[0,4],v1)
display(gb5.args[0])
display(gb5.args[1])
v1

⎛  7   2     3   5    5           3       4   2     3   2⎞
⎝z₁ ⋅z₂  - z₁ ⋅z₂ , z₁ ⋅z₃ - z₁⋅z₂ ⋅z₃, z₁ ⋅z₃  - z₂ ⋅z₃ ⎠

(z₁, z₂, z₃)

[7, 10, 11, 13, 14, 15, 17, 18, 19, 21, 22, 25]

In [212]:
rho5,lTElasticity5=elasticity(gb5)
print(rho5)
print(lTElasticity5)

4/3
[[4, 0, 0, 0, 3, 0]]


In [213]:
AC=ArchimedeanComponents(gb5)
for i,component in enumerate(AC):
    print(component)

[[0, 0, 0]]
[[0, 0, 1]]
[[0, 1, 0]]
[[0, 1, 1], [1, 0, 1], [1, 1, 1]]
[[1, 0, 0]]
[[1, 1, 0]]


In [214]:
hasAcceptableElasticity(gb5,debug=True)

S has not acceptable elasticity


----

In [123]:
gbG=groebner([x1*x2-x1**7],x2,x1)

In [124]:
gbG

             ⎛⎡    7        ⎤                             ⎞
GroebnerBasis⎝⎣- x₁  + x₁⋅x₂⎦, x₂, x₁, domain=ℤ, order=lex⎠

In [125]:
rhoG,lTElasticityG=elasticity(gbG)
print(rhoG)
print(lTElasticityG)

6
[[0, 6, 1, 0]]


In [127]:
AC_G=ArchimedeanComponents(gbG)
AC_G

[[[0, 0]], [[0, 1], [1, 1]], [[1, 0]]]

In [128]:
hasAcceptableElasticity(gbG,debug=True)

S has not acceptable elasticity


---